# Data Project

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

In [ ]:
#%pip install fredapi
#%pip install git+https://github.com/alemartinello/dstapi


#Data
from fredapi import Fred
from dstapi import DstApi

# plotting
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':'0.25','grid.linestyle':'--'})
plt.rcParams.update({'font.size': 14})

In [ ]:
pris113 = DstApi("pris113")

In [ ]:
# Table summary to get an overview of the dataset
pris113.tablesummary()

In [ ]:
#Collect the variables from the data
params = {
    'table': 'pris113',
    'format': 'BULK', # semicolon separated file
    'lang': 'en',
    'variables': [
        {'code': 'TYPE', 'values': ['INDEKS']},
        {'code': 'Tid', 'values': ['*']}, # '*' is everything
        ]
    }

CPI = pris113.get_data(params=params)

display(CPI.head(5))
display(CPI.info())

**Question 1**

In [ ]:
#Pivot the data
df = CPI.pivot_table(index='TID',columns='TYPE',values='INDHOLD', aggfunc='first')

#Change the names of the column to CPI
df.columns = ['CPI']

df.head(5)



In [ ]:
# konverter TID til datetime, så vi kan filtrere efter år
df.index = pd.to_datetime(df.index, format='%YM%m')

#Konverte CPI til numerisk
df["CPI"] = pd.to_numeric(df["CPI"], errors="coerce")

# Udregn gennemsnitligt CPI for 2020
cpi_2020 = df[df.index.year == 2020]['CPI'].mean()

# Rebase (gør 2020 = 100)
df["CPI_rebased"] = df["CPI"] / cpi_2020 * 100

In [ ]:
#Create new variables
df["inflation_monthly"] = df["CPI"] / df["CPI"].shift(1) - 1
df["inflation_yearly"] = df["CPI"] / df["CPI"].shift(12) - 1


In [ ]:
#plot CPI_rebased
plt.figure(figsize=(12,6))
plt.plot(df.index, df["CPI_rebased"], label='CPI Rebased (2020=100)', color=colors[0])
plt.title('Consumer Price Index (CPI) Rebased to 2020')
plt.xlabel('Year')
plt.ylabel('CPI (2020=100)')
plt.legend()
plt.show()

#plot inflation_monthly
plt.figure(figsize=(12,6))
plt.plot(df.index, df["inflation_monthly"]*100, label='Monthly Inflation Rate', color=colors[2])
plt.title('Monthly Inflation Rate based on CPI')
plt.xlabel('Year')
plt.ylabel('Inflation Rate (%)')
plt.legend()
plt.show()

#plot inflation_yearly
plt.figure(figsize=(12,6))
plt.plot(df.index, df["inflation_yearly"]*100, label='Yearly Inflation Rate', color=colors[1])
plt.title('Yearly Inflation Rate based on CPI')
plt.xlabel('Year')
plt.ylabel('Inflation Rate (%)')
plt.legend()
plt.show()

The yearly inflation rate was back to normal somewhere between 2023 and 2024, but the montly inflation rates were still high in 2025.

**Question 2**

In [ ]:
def func(T, K, alpha):
    return (((T - K) ** alpha) /(sum((T-K)**alpha)) * T) 

def ii_rate(P, func):
    return (sum(1 + P) ** func)-1